# Предсказываем стоимость мед страховки



## Загужаем необходимые библиотеки

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Загружаем данные

Собраны данные:
- age: возраст
- sex: пол
- bmi: индекс массы тела
- children: количество детей, охваченных медицинским страхованием / количество иждивенцев
- smoker: курение
- region: регион (northeast, southeast, southwest, northwest).
- charges: индивидуальные медицинские расходы (его и хотим предсказать)

In [ ]:
# Загрузите данные из файла insurance.csv в переменную df

In [ ]:
df = pd.read_csv('insurance.csv', sep=',')

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Смотрим статистику, что нет пропусков и отсуствующих значений

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [ ]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


## Преобразуем строковые данные

In [ ]:
# Заменяем пол и курение на числа
df['sex']=df['sex'].map({'male':1, 'female':0})
df['smoker']=df['smoker'].map({'yes':1,'no':0})

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [ ]:
# Заменяем регион на набор отдельных колонок (is_southwest, is_southeast и тп)
df = pd.get_dummies(df, columns=['region'])
df.head()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0


## Формируем признаки и целевую переменную

In [ ]:
## Сформируйте признаки и целевую переменную

In [ ]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'charges',
       'region_northeast', 'region_northwest', 'region_southeast',
       'region_southwest'],
      dtype='object')

In [48]:
X = df[['age', 'sex', 'bmi', 'children', 'smoker', 'region_northeast', 'region_northwest', 'region_southeast','region_southwest']]
y = df['charges']

In [49]:
X

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,0,0,0,1
1,18,1,33.770,1,0,0,0,1,0
2,28,1,33.000,3,0,0,0,1,0
3,33,1,22.705,0,0,0,1,0,0
4,32,1,28.880,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,0,1,0,0
1334,18,0,31.920,0,0,1,0,0,0
1335,18,0,36.850,0,0,0,0,1,0
1336,21,0,25.800,0,0,0,0,0,1


In [ ]:
y

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1338, dtype: float64

## Разделяем данные на выборку для обучения/проверки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
983,27,0,30.590,1,0,1,0,0,0
1209,59,1,37.100,1,0,0,0,0,1
957,24,1,26.790,1,0,0,1,0,0
1297,28,0,26.510,2,0,0,0,1,0
27,55,0,32.775,2,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1239,25,0,42.130,1,0,0,0,1,0
77,21,1,35.530,0,0,0,0,1,0
138,54,0,31.900,3,0,0,0,1,0
1114,23,1,24.510,0,0,1,0,0,0


In [ ]:
X_test

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
1334,18,0,31.920,0,0,1,0,0,0
515,58,1,35.700,0,0,0,0,0,1
1324,31,1,25.935,1,0,0,1,0,0
1082,38,1,19.950,1,0,0,1,0,0
1274,26,1,27.060,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...
493,61,1,43.400,0,0,0,0,0,1
1323,42,0,40.370,2,1,0,0,1,0
802,21,1,22.300,1,0,0,0,0,1
76,29,0,29.590,1,0,0,0,1,0


In [ ]:
y_train

983     16796.41194
1209    12347.17200
957     12609.88702
1297     4340.44090
27      12268.63225
           ...     
1239     3238.43570
77       1532.46970
138     27322.73386
1114     2396.09590
374      1391.52870
Name: charges, Length: 1070, dtype: float64

In [ ]:
y_test

1334     2205.98080
515     11362.75500
1324     4239.89265
1082     5855.90250
1274    17043.34140
           ...     
493     12574.04900
1323    43896.37630
802      2103.08000
76       3947.41310
94      47291.05500
Name: charges, Length: 268, dtype: float64

## Создаем и обучаем модель линейной регресии

In [ ]:
# Создайте и обучите модель

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression() 

In [ ]:
lr.fit(X_train, y_train)

LinearRegression()

In [ ]:
lr.coef_

array([  256.78132509,  -157.51791799,   327.6390977 ,   522.4535018 ,
       23661.70879536,   665.90137211,   324.51766919,  -749.85800286,
        -240.56103843])

## Получаем предсказание и оцениваем качество

In [ ]:
# Получите предсказание

In [ ]:
lr.predict(X_test)

array([ 3509.06761441, 13954.8160788 ,  4909.85672175,  4746.40599766,
       26059.42370271, 12855.30068368, 26880.91021104,  8213.65444534,
       37398.07849021, 12583.27763017, 31824.23729789, 39002.63673305,
        8734.61725382,  6963.0908176 ,  5955.71694679, 11647.46999034,
        8915.04423271, 11114.64135626, 28901.74900849,  -341.56312561,
       36991.67080409, 11853.33666254, 12425.38473932, 14756.97507557,
        1839.0059286 ,  6941.91900166,  6247.0773823 , 12358.95480124,
       12880.90453961,  5466.4405252 ,  9726.73931066, 35073.64330754,
       11685.20953283,  3723.22532074,  8748.92850931,  3501.12495454,
       25307.12314874, 14708.3860035 ,  8731.2671109 , 25164.24817302,
        5433.94447987, 33588.36900187, 27377.84829388, 11638.26671349,
       15044.08350485, 13508.96395451, 12218.3187882 , 32019.47790184,
        5128.55812134,  3888.91143941, 15263.72862016, 11578.51164987,
        1057.02160461, 10358.65298388,  7768.58159939, 11304.92450479,
      

In [ ]:
# Оцените качество, при помощи метода mean_squared_error для тестовой выборки

In [ ]:
pd.DataFrame([y_test.values, lr.predict(X_test)]).T

,0,1
0,2205.98080,3509.067614
1,11362.75500,13954.816079
2,4239.89265,4909.856722
3,5855.90250,4746.405998
4,17043.34140,26059.423703
...,...,...
263,12574.04900,17247.981106
264,43896.37630,35731.226216
265,2103.08000,585.996643
266,3947.41310,4676.957220


In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_test, lr.predict(X_test))

39827456.529814795

In [ ]:
mean_squared_error(y_train, lr.predict(X_train))

35746074.82947608

## Делаем предсказание для одного человека

In [ ]:
# Заполняем данные по конкретному человеку
data = [{
    "age": 20,
    "sex": 1,
    "bmi": 30,
    "children": 2,
    "smoker": 1,
    "region_northeast": 0,
    "region_northwest": 0,
    "region_southeast": 1,
    "region_southwest": 0
}]

In [ ]:
df_person = pd.DataFrame(data)
df_person.head()

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,20,1,30,2,1,0,0,1,0


In [ ]:
lr.predict(df_person)

array([26526.90170301])